In [1]:
VERSION = "cv-corpus-6.1-2020-12-11"
LANGUAGE = "en"

In [2]:
import librosa
import argparse
import csv
import json
import logging
import multiprocessing
import os
import subprocess
import sys
import tarfile
from multiprocessing.pool import ThreadPool
from pathlib import Path
from typing import List
import yaml

from tqdm import tqdm
import pandas as pd
import numpy as np

In [5]:
path ="/Users/dami.osoba/work/bawk/small_dataset/small/CV_unpacked/cv-corpus-6.1-2020-12-11/en/validated.tsv"
validated = pd.read_csv(path,sep='\t')

In [6]:
fourth_pd = validated[validated['sentence'].astype('str')\
                      .apply(lambda x : any(a_string in x.lower().split() for a_string in ['fourth','brain','hey']) or (len(x.split())<5))]

In [27]:
ln = validated.shape[0]
train_ix = np.random.choice(ln,1000,replace=False)
idx2 = validated.index.difference(train_ix)
test_ix = np.random.choice(idx2,200,replace=False)
idx3 = validated.index.difference(np.concatenate([test_ix,train_ix]))
val_ix = np.random.choice(idx3,100,replace=False)


In [8]:
parser = argparse.ArgumentParser(description='Downloads and processes Mozilla Common Voice dataset.')
parser.add_argument("--data_root", default='./', type=str, help="Directory to store the dataset.")
parser.add_argument('--manifest_dir', default='./', type=str,help='Output directory for manifests')
parser.add_argument("--num_workers", default=multiprocessing.cpu_count(), type=int, help="Workers to process dataset.")
parser.add_argument('--sample_rate', default=16000, type=int, help='Sample rate')
parser.add_argument('--n_channels', default=1, type=int, help='Number of channels for output wav files')

sys.argv = ['-f']
# args = parser.parse_args(['--data_root','/home/ubuntu/cv-corpus-6.1-2020-12-11/en',
#                           '--manifest_dir','/home/ubuntu/cv-corpus-6.1-2020-12-11/en'])

args = parser.parse_args(['--data_root','/Users/dami.osoba/work/cv-corpus-6.1-2020-12-11/en',
                          '--manifest_dir','/Users/dami.osoba/work/cv-corpus-6.1-2020-12-11/en'])





def create_manifest(data: List[tuple], output_name: str, manifest_path: str):
    output_file = Path(manifest_path) / output_name
    output_file.parent.mkdir(exist_ok=True, parents=True)

    with output_file.open(mode='w') as f:
        for wav_path, duration, text in tqdm(data, total=len(data)):
            if wav_path:
              f.write(
                  json.dumps({'audio_filepath': os.path.abspath(wav_path), "duration": duration, 'text': text}) + '\n'
              )


def process_files(csv_file, data_root, num_workers):
    """ Read *.csv file description, convert mp3 to wav, process text.
        Save results to data_root.
    Args:
        csv_file: str, path to *.csv file with data description, usually start from 'cv-'
        data_root: str, path to dir to save results; wav/ dir will be created
    """
#     wav_dir = os.path.join(data_root, 'fourth_small_wav/')
    wav_dir = '/Users/dami.osoba/work/bawk/fourth_smal_wav'
    os.makedirs(wav_dir, exist_ok=True)
#     audio_clips_path = "/home/ubuntu/cv-corpus-6.1-2020-12-11/en/clips/"
    audio_clips_path = os.path.dirname(csv_file) + '/clips/'


    def process(x):
        file_path, text = x
        file_name = os.path.splitext(os.path.basename(file_path))[0]
        text = text.lower().strip()
        audio_path = os.path.join(audio_clips_path, file_path)
        print(audio_path)
        output_wav_path = os.path.join(wav_dir, file_name + '.wav')
        tfm = Transformer()
        tfm.rate(samplerate=args.sample_rate)
        tfm.channels(n_channels=args.n_channels)
        os.system(
        f'ffmpeg -i {audio_path} -ar {args.sample_rate} {output_wav_path} -y'
    )
        print(output_wav_path)
        duration = librosa.get_duration(filename=output_wav_path)
        return output_wav_path, duration, text


    logging.info('Converting mp3 to wav for {}.'.format(csv_file))
#     with open(csv_file) as csvfile:
#         reader = csv.DictReader(csvfile, delimiter='\t')
#         next(reader, None)  # skip the headers     
#         data = [(row['path'], row['sentence']) for row in reader if row in curated] 
    data = [(row['path'], row['sentence']) for row in fourth_pd] 

    with ThreadPool(num_workers) as pool:
        data = list(tqdm(pool.imap(process, data), total=len(data)))
    return data


def main():
    data_root = args.data_root
    os.makedirs(data_root, exist_ok=True)

    target_unpacked_dir = os.path.join(data_root, "CV_unpacked")
    logging.info('Find existing folder {}'.format(target_unpacked_dir))


    folder_path = '/Users/dami.osoba/work/cv-corpus-6.1-2020-12-11/en'

    for csv_file in ['validated.tsv']:
        data = process_files(
            csv_file=os.path.join(folder_path, csv_file),
            data_root=os.path.join(data_root, csv_file.split('.')[0]),
            num_workers=args.num_workers,
        )
        logging.info('Creating manifests...')
        create_manifest(
            data=data,
#             output_name=f'commonvoice_{os.path.splitext(csv_file)[0]}_manifest.json',
            output_name=f'commonvoice_fourth_small_manifest.json',
            manifest_path=args.manifest_dir,
        )


if __name__ == "__main__":
    main()

TypeError: string indices must be integers

In [24]:
parser = argparse.ArgumentParser(description='Downloads and processes Mozilla Common Voice dataset.')
parser.add_argument("--data_root", default='./', type=str, help="Directory to store the dataset.")
parser.add_argument('--manifest_dir', default='./', type=str,help='Output directory for manifests')
parser.add_argument("--num_workers", default=multiprocessing.cpu_count(), type=int, help="Workers to process dataset.")
parser.add_argument('--sample_rate', default=16000, type=int, help='Sample rate')
parser.add_argument('--n_channels', default=1, type=int, help='Number of channels for output wav files')

sys.argv = ['-f']
# args = parser.parse_args(['--data_root','/home/ubuntu/cv-corpus-6.1-2020-12-11/en',
#                           '--manifest_dir','/home/ubuntu/cv-corpus-6.1-2020-12-11/en'])

args = parser.parse_args(['--data_root','/Users/dami.osoba/work/cv-corpus-6.1-2020-12-11/en',
                          '--manifest_dir','/Users/dami.osoba/work/cv-corpus-6.1-2020-12-11/en'])





def create_manifest(data: List[tuple], output_name: str, manifest_path: str):
    output_file = Path(manifest_path) / output_name
    output_file.parent.mkdir(exist_ok=True, parents=True)

    with output_file.open(mode='w') as f:
        for wav_path, duration, text in tqdm(data, total=len(data)):
            if wav_path:
              f.write(
                  json.dumps({'audio_filepath': os.path.abspath(wav_path), "duration": duration, 'text': text}) + '\n'
              )


def process_files(num_workers):
    """ Read *.csv file description, convert mp3 to wav, process text.
        Save results to data_root.
    Args:
        csv_file: str, path to *.csv file with data description, usually start from 'cv-'
        data_root: str, path to dir to save results; wav/ dir will be created
    """
    wav_dir = '/Users/dami.osoba/work/bawk/fourth_small_wav'
    os.makedirs(wav_dir, exist_ok=True)
    audio_clips_path = "/Users/dami.osoba/work/cv-corpus-6.1-2020-12-11/en/clips"


    def process(x):
        file_path, text = x
        file_name = os.path.splitext(os.path.basename(file_path))[0]
        text = str(text).lower().strip()
        audio_path = os.path.join(audio_clips_path, file_path)
        output_wav_path = os.path.join(wav_dir, file_name + '.wav')
        os.system(
        f'ffmpeg -i {audio_path} -ar {args.sample_rate} {output_wav_path} -y'
    )
        duration = librosa.get_duration(filename=output_wav_path)
        return output_wav_path, duration, text


    logging.info('Converting mp3 to wav for')
    data = [(row[1]['path'], row[1]['sentence']) for row in fourth_pd.iterrows()]
    with ThreadPool(num_workers) as pool:
        data = list(tqdm(pool.imap(process, data), total=len(data)))
    return data


def main():
    data_root = args.data_root
    os.makedirs(data_root, exist_ok=True)

    target_unpacked_dir = os.path.join(data_root, "CV_unpacked")
    logging.info('Find existing folder {}'.format(target_unpacked_dir))


    folder_path = '/Users/dami.osoba/work/cv-corpus-6.1-2020-12-11/en'
    csv_file = os.path.join(folder_path, "validated.tsv")
    data = process_files(num_workers=args.num_workers)
    logging.info('Creating manifests...')
    create_manifest(
        data=data,
        output_name=f'commonvoice_fourth_small_manifest.json',
        manifest_path=args.manifest_dir,
        )


if __name__ == "__main__":
    main()

 98%|█████████▊| 124913/127103 [1:44:47<01:49, 20.04it/s]  

In [16]:
next(fourth_pd.iterrows())[1]['path']

'common_voice_en_699711.mp3'

In [ ]:
# working directly in ec2
parser = argparse.ArgumentParser(description='Downloads and processes Mozilla Common Voice dataset.')
parser.add_argument("--data_root", default='./', type=str, help="Directory to store the dataset.")
parser.add_argument('--manifest_dir', default='./', type=str,help='Output directory for manifests')
parser.add_argument("--num_workers", default=multiprocessing.cpu_count(), type=int, help="Workers to process dataset.")
parser.add_argument('--sample_rate', default=16000, type=int, help='Sample rate')
parser.add_argument('--n_channels', default=1, type=int, help='Number of channels for output wav files')

sys.argv = ['-f']
# args = parser.parse_args(['--data_root','/home/ubuntu/cv-corpus-6.1-2020-12-11/en',
#                           '--manifest_dir','/home/ubuntu/cv-corpus-6.1-2020-12-11/en'])




def create_manifest(data: List[tuple], output_name: str, manifest_path: str):
    output_file = Path(manifest_path) / output_name
    output_file.parent.mkdir(exist_ok=True, parents=True)

    with output_file.open(mode='w') as f:
        for wav_path, duration, text in tqdm(data, total=len(data)):
            if wav_path:
              f.write(
                  json.dumps({'audio_filepath': os.path.abspath(wav_path), "duration": duration, 'text': text}) + '\n'
              )


def process_files(csv_file, data_root, num_workers):
    """ Read *.csv file description, convert mp3 to wav, process text.
        Save results to data_root.
    Args:
        csv_file: str, path to *.csv file with data description, usually start from 'cv-'
        data_root: str, path to dir to save results; wav/ dir will be created
    """
#     wav_dir = os.path.join(data_root, 'wav/')
    wav_dir = "/home/ubuntu/validated"
    os.makedirs(wav_dir, exist_ok=True)
#     audio_clips_path = "/home/ubuntu/cv-corpus-6.1-2020-12-11/en/clips/"
    audio_clips_path = os.path.dirname(csv_file) + '/clips/'


    def process(x):
        file_path, text = x
        file_name = os.path.splitext(os.path.basename(file_path))[0]
        text = text.lower().strip()
        audio_path = os.path.join(audio_clips_path, file_path)
        output_wav_path = os.path.join(wav_dir, file_name + '.wav')
        tfm = Transformer()
        tfm.rate(samplerate=args.sample_rate)
        tfm.channels(n_channels=args.n_channels)
        os.system(
        f'ffmpeg -i {audio_path} -ar {args.sample_rate} {output_wav_path} -y'
    )
        duration = librosa.get_duration(filename=output_wav_path)
        return output_wav_path, duration, text


    logging.info('Converting mp3 to wav for {}.'.format(csv_file))
    with open(csv_file) as csvfile:
        reader = csv.DictReader(csvfile, delimiter='\t')
        next(reader, None)  # skip the headers
        data = [(row['path'], row['sentence']) for row in reader] 
        with ThreadPool(num_workers) as pool:
            data = list(tqdm(pool.imap(process, data), total=len(data)))
    return data


def main():
    data_root = args.data_root
    os.makedirs(data_root, exist_ok=True)

    target_unpacked_dir = os.path.join(data_root, "CV_unpacked")
    logging.info('Find existing folder {}'.format(target_unpacked_dir))


    folder_path = '/home/ubuntu/cv-corpus-6.1-2020-12-11/en'

    for csv_file in ['validated.tsv']:
        data = process_files(
            csv_file=os.path.join(folder_path, csv_file),
            data_root=os.path.join(data_root, csv_file.split('.')[0]),
            num_workers=args.num_workers,
        )
        logging.info('Creating manifests...')
        create_manifest(
            data=data,
            output_name=f'commonvoice_{os.path.splitext(csv_file)[0]}_manifest.json',

            manifest_path=args.manifest_dir,
        )


if __name__ == "__main__":
    main()